In [32]:
import numpy as np
import math
from PIL import Image
import cv2

In [2]:
def get_as_png(file_name):
    '''Converts the input image to PNG format.'''
    image = Image.open(file_name)
    image.save('image-to-classify.png')
    image = Image.open('image-to-classify.png')
    return image

In [3]:
def trim_edges(image):
    '''Trims edges that are close to the color of the corner pixels.
       The closeness is defined by the distance within 5% of the maximum
       possible intensity in RGB space.
    '''
    image_data = np.asarray(image)
    # upper left corner
    corner_pixel_1 = image_data[0, 0]
    # upper right corner
    corner_pixel_2 = image_data[0, image_data.shape[1] - 1]
    # lower right corner
    corner_pixel_3 = image_data[image_data.shape[0] - 1, image_data.shape[1] - 1]
    # lower left corner
    corner_pixel_4 = image_data[image_data.shape[0] - 1, 0]
    
    pass

In [82]:
def crop(file_name):
    im = cv2.imread(file_name)
    imgray = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    imgray = cv2.blur(imgray,(15,15))
    ret,thresh = cv2.threshold(imgray,math.floor(np.average(imgray)),255,cv2.THRESH_BINARY_INV)
    dilated = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(10,10)))
    _, contours, _ = cv2.findContours(dilated,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    
    height, width, channels = im.shape
    
    new_contours = []
    for c in contours:
        if cv2.contourArea(c) < (width * height):
            new_contours.append(c)
    
    best_box=[-1, -1, -1, -1]
    '''index = 0
    for c in new_contours:
        print("best:", best_box)
        x,y,w,h = cv2.boundingRect(c)
        print(x, y)
        print(w, h)
        print("index:", index)
        index += 1
        if (x + w) == width and (y + h) == height:
            continue
        if best_box[0] < 0:
            best_box=[x,y,x+w,y+h]
        else:
            if x<best_box[0]:
                best_box[0]=x
            if y<best_box[1]:
                best_box[1]=y
            if x+w>best_box[2]:
                best_box[2]=x+w
            if y+h>best_box[3]:
                best_box[3]=y+h'''
    best_box = [-1, -1, -1, -1]
    for c in new_contours:
        x,y,w,h = cv2.boundingRect(c)
        if (w * h) > (best_box[2] * best_box[3]):
            best_box = [x, y, w, h]
    print(best_box)
    final_img = im[best_box[1]:best_box[3], best_box[0]:best_box[2]]
    cv2.imwrite('result.png', final_img)

In [78]:
def resize_longest_edge(image):
    '''Resizes the longest edge to 28 pixels by subsampling the pixels and
       scales the shorter edge accordingly.
    '''
    # remember to scale the shortest edge
    pass

In [64]:
def sharpen_pixels(image):
    '''Sharpens pixels using a Gaussian operator of the radius and
       standard deviation of 1.0.
    '''
    pass

In [65]:
def resize_shortest_edge(image):
    '''Extends the shortest edge to 28 pixels and centers the new image.'''
    pass

In [66]:
def negate_intensities(image):
    '''Negates the intensities of pixels.'''
    pass

In [67]:
def convert_to_grayscale(image):
    '''Converts given pixels to 8-bit grayscale pixels.'''
    pass

In [68]:
#png = get_as_png('test-shirt2.jpg')

In [69]:
#trimmed_edges = trim_edges(png)

In [83]:
crop('test-shirt4.jpg')

[0, 556, 2448, 2708]


In [16]:
cropped_image = Image.open('cropped_image.png')

FileNotFoundError: [Errno 2] No such file or directory: 'cropped_image.png'

In [9]:
image_data = np.asarray(image)
image_data.shape

(3264, 2448, 3)

In [36]:
last_column = image_data.shape[1]
last_column

2448

In [32]:
image_data

array([[[176, 151, 129],
        [172, 147, 125],
        [172, 145, 124],
        ...,
        [200, 166, 139],
        [195, 161, 134],
        [190, 156, 129]],

       [[170, 145, 123],
        [169, 144, 122],
        [170, 143, 122],
        ...,
        [197, 163, 136],
        [195, 161, 134],
        [194, 160, 133]],

       [[170, 145, 123],
        [170, 145, 123],
        [172, 145, 124],
        ...,
        [195, 161, 134],
        [196, 162, 135],
        [199, 165, 138]],

       ...,

       [[123,  85,  74],
        [123,  87,  75],
        [135,  99,  87],
        ...,
        [252, 226, 177],
        [253, 227, 178],
        [254, 228, 179]],

       [[128,  90,  79],
        [127,  91,  79],
        [133,  97,  85],
        ...,
        [252, 226, 177],
        [252, 226, 177],
        [252, 226, 177]],

       [[133,  95,  84],
        [127,  91,  79],
        [127,  91,  79],
        ...,
        [253, 227, 178],
        [251, 225, 176],
        [251, 225, 176]]

In [34]:
first_pixel = image_data[0, 0]
first_pixel

array([176, 151, 129], dtype=uint8)

In [12]:
start_index = 0
len(image_data)

2448

In [ ]:
resized = image_data[start_index::step_size]